In [1]:
import os
import sys
current_path = os.path.abspath(os.getcwd())
PROJECT_DIR = os.path.abspath("../..")
print(PROJECT_DIR)
os.chdir(PROJECT_DIR)
sys.path.append(PROJECT_DIR)
from joblib import Parallel, delayed
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

/home/S22/workspace/lyj/BasicTS250828


In [2]:
from typing import Dict
import torch
from arch import arch_model
from torch.utils.data import DataLoader
import numpy as np
from basicts.utils import load_pkl, get_regular_settings
from basicts.data import TimeSeriesForecastingDataset
from basicts.metrics import masked_mae, masked_rmse, masked_mape, masked_mse
from basicts.scaler import ZScoreScaler
import pandas as pd

## Hyper-parameters

In [ ]:
# construct configs
dataset_name = "ETTh1"

regular_settings = get_regular_settings(dataset_name)

input_len = 96#regular_settings['INPUT_LEN']
output_len = 96#regular_settings['OUTPUT_LEN']
rescale = regular_settings['RESCALE']
null_val = regular_settings['NULL_VAL']
norm_each_channel = regular_settings['NORM_EACH_CHANNEL']
train_val_test_ratio = regular_settings['TRAIN_VAL_TEST_RATIO']

target_time_series = None # for subset forecasting

gpu_num = 1
batch_size = 128 # only used for collecting data

# GARCH params
# Refer to https://pypi.org/project/arch/ for parameter settings. We only provide a basic runtime framework.
params = {
    'vol': 'GARCH', 
    'p': 1,
    'q': 1
}

## Construct Dataset

In [4]:
test_set = TimeSeriesForecastingDataset(dataset_name=dataset_name, input_len=input_len, output_len=output_len, train_val_test_ratio=train_val_test_ratio, mode="test")
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

scaler = ZScoreScaler(dataset_name=dataset_name, train_ratio=train_val_test_ratio[0], norm_each_channel=norm_each_channel, rescale=rescale)

In [5]:
Xs_test = []
Ys_test = []

def preprocessing(input_data, scaler, target_time_series) -> Dict:
    if scaler is not None:
        input_data['target'] = scaler.transform(input_data['target'])
        input_data['inputs'] = scaler.transform(input_data['inputs'])
    if target_time_series is not None:
        input_data['target'] = input_data['target'][:, :, target_time_series, :]
        input_data['inputs'] = input_data['inputs'][:, :, target_time_series, :]
    return input_data

# Local forecasting
# Test dataset only
for i, iter_data in enumerate(test_loader):
    iter_data = preprocessing(iter_data, scaler=scaler, target_time_series=target_time_series)
    inputs, target = iter_data['inputs'], iter_data['target']
    Xs_test.append(inputs)
    Ys_test.append(target)


Xs_test = torch.cat(Xs_test, dim=0)[..., [0]]
Xs_test = Xs_test[::input_len,:,:,:] # The time complexity of ARCH is so high that we have to sample at intervals to test its performance.
Ys_test = torch.cat(Ys_test, dim=0)[..., [0]]
Ys_test = Ys_test[::input_len,:,:,:]

In [6]:
def reshape(data):
    B, L, N, C = data.shape
    data = data[..., 0].transpose(1, 2)
    return data

In [7]:
B, N, L = reshape(Xs_test).shape
B, N, L2 = reshape(Ys_test).shape
result = np.zeros((B, N, L2))
Xs_test = reshape(Xs_test).numpy()

## Train (Direct Multi-Step Forecasting)

In [ ]:
# Fitting and Forecasting Functions
def fit_and_forecast(args, params):
    i, j, data, real = args
    # Fitting
    estimator = arch_model(data * 10, **params).fit(disp='off') # Ensure your values generally fall between 1 and 1000 due to ARCH's code limitations.
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    # Forecasting
    forecast_result = estimator.forecast(horizon=L2)
    forecast = forecast_result.mean.values[-1, :] / 10
    
    return i, j, forecast


print(Xs_test.shape)
# Create task list
tasks = [(i, j,  Xs_test[i, j, :], Ys_test[i, j, :]) for i in range(B) for j in range(N)]
with Parallel(n_jobs=-1) as parallel:
    results = list(parallel(delayed(fit_and_forecast)(task, params) for task in tasks))


(29, 7, 96)


## Test (Direct Multi-Step Forecasting)

In [9]:
Ys_test = reshape(Ys_test)
B, N, L = Ys_test.shape
prediction = torch.tensor(result).reshape(B, N, L)
real_value = Ys_test

In [10]:
# print results
print("MAE: ", masked_mae(prediction, real_value, null_val).item())
print("RMSE: ", masked_rmse(prediction, real_value, null_val).item())
print("MSE: ", masked_mse(prediction, real_value, null_val).item())
print("MAPE: {:.2f}%".format(masked_mape(prediction, real_value, null_val) * 100))

MAE:  0.796024530010228
RMSE:  1.0522329139423747
MSE:  1.107194105183661
MAPE: 100.00%
